In [1]:
from elasticsearch import Elasticsearch
import os
import glob
import PyPDF2
import pandas as pd

In [2]:
os.chdir("C:\\tmp\\jp")
files=glob.glob("*.*")

In [3]:
len(files)

5

In [4]:
for catalog in files:
    print(catalog)

ACFrOgAHo4sXcSv5qAOuMBr0uEWAOKGCh5NigeYzfP7TjOf7nNJFi7eP4Bjlfp9yGFyB7PiQCcsnNGA7e9AH1_LtGrKgCqCWqmsElMEw20u5aR0gDdBIX9_y36-cpY0=.pdf
ACFrOgAI3Y336GlEyJBySMpJQbsxFJY4OLPbBLCV53g7eljlGnyrt2l3B3BApP8Dclnpdpk5Mu8Kn2usHrOzzoHQGXO4HmvgjqWtQLCHsfBEuZPNA1Nrf7ufW-NlDHQ=.pdf
ACFrOgB-HyyouVKN_pDn9PD5SPsjIIurmD5B9iB-j0FSKTowpvV1bCQnR9mTT8Y8y59w6rR-0qGNWZVPCRXJi2s-ysapNxHp8mpRBvDSX_A5waXIsH8ndVx6OphWYAo=.pdf
ACFrOgBJW4oArIaLPE2RJ3VdYwNQnVw__wzSEzQlMnYhY1jbqlYAT9XpGaORTL7Kfnbqiu2Tpez-LLdGzbQ5HRP8izvROPCyECyXdbILE9boAzDiLiprCU9h3qvEDDo=.pdf
ACFrOgBWZenYp9goTfOZ-StQJqg2d72CeQh-GPM8uCHqWE2rHpE1PAXWXsnO9irrvvM7ZiifwP8XfZQEqnpQ-dDh8x74MjnHGsEWgaCRBR41L-TN54i2rrjR1Gxyn7E=.pdf


In [5]:
def extractPdfFiles(files):
    this_loc=1
    df=pd.DataFrame(columns=("name","content"))
    
    for file in files:
        pdfFileObj=open(file,'rb')
        pdfReader=PyPDF2.PdfFileReader(pdfFileObj)
        n_pages=pdfReader.numPages
        this_doc=''
        for i in range(n_pages):
            pageObj=pdfReader.getPage(i)
            this_text=pageObj.extractText()
            this_doc+=this_text
        df.loc[this_loc]=file,this_doc
        this_loc=this_loc+1
    return df

In [6]:
df=extractPdfFiles(files)

In [7]:
df.head()

,name,content
1,ACFrOgAHo4sXcSv5qAOuMBr0uEWAOKGCh5NigeYzfP7TjO...,23\n610203036\n44\n\n48\n52\n\n586470\n2102\n2...
2,ACFrOgAI3Y336GlEyJBySMpJQbsxFJY4OLPbBLCV53g7el...,\n\n\n\n\n\n\n\nNOXAAAA\n H18-006T1\n\n\n\n\n\...
3,ACFrOgB-HyyouVKN_pDn9PD5SPsjIIurmD5B9iB-j0FSKT...,\n\n\n\n\n\n˛7˛7˛7˛7˛7˛7˛7˛7˛7˛7˛7˛7˛7˛7˛7˛7˛7...
4,ACFrOgBJW4oArIaLPE2RJ3VdYwNQnVw__wzSEzQlMnYhY1...,"Shall we open, AP Windowby YKK AP\n\n\n\n\n3\n..."
5,ACFrOgBWZenYp9goTfOZ-StQJqg2d72CeQh-GPM8uCHqWE...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [8]:
es=Elasticsearch()

In [9]:
col_names=df.columns
for row_number in range(df.shape[0]):
    body=dict([(name,str(df.iloc[row_number][name])) for name in col_names])
    es.index(index='catalogs',doc_type='doc',body=body)

In [32]:
search_results=es.search(index='catalogs',doc_type='doc',
                        body={"_source":"name",
                             "query":{
                                 "match_phrase":{"content":"掲載内容"},
                             }
                        })

In [33]:
search_results

{'took': 11,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 1,
  'max_score': 1.8330884,
  'hits': [{'_index': 'catalogs',
    '_type': 'doc',
    '_id': '458fc5922a961f24ec68d86b9cdfae7',
    '_score': 1.8330884,
    '_source': {}}]}}

In [34]:
search_results['hits']['total']

1